PyImageJ Code Snippets
===

In this notebook we'll cover how to perform some common imagej operations (_i.e._ extracting channels, merging, etc..) in PyImageJ.

First let's intitalize ImageJ and load in a test stack (3-channel time series dataset).

In [1]:
import imagej
import scyjava as sj
import numpy as np
import xarray as xr

# start imagej
ij = imagej.init()

# load image
path = "/home/edward/Documents/scratch/data/test.tif"
img = ij.io().open(path)

## Extract channels

Channels can be extracted by slicing the `xarray` channel coordinate. Below is a simple method that simplifies this process. Here you just need to pass the stack and which channel you want extracted.

In [2]:
def extract_channel(stack, channel: int):
    channel_num = channel - 1

    if len(stack.dims) == 4:
        extract = stack[:,channel_num,:,:]
        print("Extracted channel {}.".format(str(channel)))
    elif len(stack.dims) == 3:
        extract = stack[:,:,channel_num]
    else:
        print("No channels found: {}".format(stack.dims))

    # re-attach the Channel coordinate with the Channel dimension
    extract = extract.expand_dims("Channel")
    extract = extract.rename("Channel " + str(channel) )

    return extract

In [3]:
# convert dataset to xarray
img_xarr = ij.py.from_java(img)

# extract each channel
img_ch1 = extract_channel(img_xarr, 1)
img_ch2 = extract_channel(img_xarr, 2)
img_ch3 = extract_channel(img_xarr, 3)

Extracted channel 1.
Extracted channel 2.
Extracted channel 3.


## Combine channels

Split channels can be merged by using `xarray.merge()`. Below we merge channel 1 and channel 3 and create a new xarray `Dataset`.

In [4]:
# combine channel 1 and 3
img_combo = xr.merge([img_ch1, img_ch3])

# print new xarray dataset
img_combo

<xarray.Dataset>
Dimensions:    (Channel: 2, Time: 83, x: 300, y: 300)
Coordinates:
  * Channel    (Channel) float64 0.0 2.0
  * x          (x) float64 0.0 0.325 0.65 0.975 1.3 ... 96.2 96.53 96.85 97.18
  * y          (y) float64 0.0 0.325 0.65 0.975 1.3 ... 96.2 96.53 96.85 97.18
  * Time       (Time) float64 0.0 1.0 2.0 3.0 4.0 ... 78.0 79.0 80.0 81.0 82.0
Data variables:
    Channel 1  (Channel, Time, y, x) float64 1.132e+03 1.064e+03 ... nan nan
    Channel 3  (Channel, Time, y, x) float64 nan nan nan ... 2.534e+04 2.594e+04